*Step.1 導入相關套件*

In [ ]:
from huggingface_hub import notebook_login
notebook_login("")

In [ ]:
!pip install datasets

In [ ]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

*Step.2 載入數據*

In [ ]:
ds = Dataset.load_from_disk("./alpaca_data_zh/")
ds

In [ ]:
ds[:3]

*Step.3 數據前處理*

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")
tokenizer

LlamaTokenizerFast(name_or_path='meta-llama/Llama-2-7b-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [ ]:
# tokenizer.padding_side='right'

In [ ]:
tokenizer.pad_token_id = 2

In [ ]:
def process_func(example):
    MAX_LENGTH = 1024 # Llama會將中文切成多個token,因此需要增大長度,確保數據完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: " + example["instruction"], example["input"]]).strip() + "\n\nAssistant: " , add_special_tokens=False)
    response = tokenizer(example["output"], add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.eos_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.eos_token_id]
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [ ]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

In [ ]:
tokenizer.decode(tokenized_ds[1]["input_ids"])

In [ ]:
# tokenizer("哈" , add_special_tokens=False) # Llama會將中文切成多個token,因此需要增大長度,確保數據完整性

{'input_ids': [29871, 232, 150, 139], 'attention_mask': [1, 1, 1, 1]}

In [ ]:
tokenizer.decode(list(filter(lambda x: x != -100, tokenized_ds[1]["labels"])))

*Step.4 模型建立*

In [ ]:
import torch
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-hf" , low_cpu_mem_usage=True , torch_dtype = torch.half , device_map = "auto")

config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
model.dtype

torch.float16

In [ ]:
for name, parameter in model.named_parameters():
    print(name , parameter.dtype)

# Lora
# PEFT Step1 配置文件

In [ ]:
from peft import LoraConfig, TaskType, get_peft_model

config = LoraConfig(task_type=TaskType.CAUSAL_LM,)
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules=None, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

# PEFT Step2 建立模型

In [ ]:
model = get_peft_model(model, config)

In [ ]:
config

LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='meta-llama/Llama-2-7b-hf', revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, r=8, target_modules={'v_proj', 'q_proj'}, lora_alpha=8, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=False, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))

In [ ]:
for name, parameter in model.named_parameters():
    print(name)

In [ ]:
# model.enable_input_require_grads() <-- 如果在args要用gradient_checkpoint要執行

In [ ]:
# model = model.half() 全模型fp16 , args中adam_epsilon要設定大一點

In [ ]:
# 觀察模型正向傳播的收斂情況
# from torch.utils.data import DataLoader
# dl = DataLoader(tokenized_ds , batch_size = 2 ,collate_fn = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True))
# ipt = next(enumerate(dl))[1]
# ipt

In [ ]:
model.print_trainable_parameters()

trainable params: 4,194,304 || all params: 6,742,609,920 || trainable%: 0.0622


*Step.5 訓練參數*

In [ ]:
args = TrainingArguments(
    output_dir = "./chatbot",
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 8,
    logging_steps = 10,
    num_train_epochs = 1,
    save_steps = 20,
    # adam_epsilon = 1e-4  # 全模型fp16,args中adam_epsilon要設定大一點
)

*Step.6 訓練器*

In [ ]:
trainer = Trainer(
    model = model,
    args = args,
    train_dataset=tokenized_ds.select(range(6000)),
    data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),

)

*Step.7 模型訓練*

In [ ]:
trainer.train()

*Step8 模型推理*

In [ ]:
model.eval()
ipt = tokenizer("Human: {}\n{}".format("你好", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(model.device)
tokenizer.decode(model.generate(**ipt, max_length=512, do_sample=True, eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True)